In [2]:
import numpy as np
import pandas as pd

### $\int_{-\inf}^{z} e^{\frac{-u^2}{2}}du = \frac{1-0.05}{2}*\sqrt{2*π}$
(обчислення $z_\gamma$ із лекції 7, 2 ст)

In [3]:
z_gamma_empty = 0.062706777

### Завдання 1

Перевірка однорідності (критерій пустих блоків)

Маємо дві незалежні вибірки: $\overline{X} = (X_1, \cdots, X_n), \overline{Y} = (Y_1, \cdots, Y_n)$
1. Розбиваємо вісь $R$ на напівінтервали $(X_{(i-1)}, X_{(i)}], i=1, \cdot, n+1$
2. Обчислюємо кількість пустих блоків: $k = # \text{пустих блоків}$
3. $Crit = \dfrac{n}{1+\rho} + \sqrt{n} \dfrac{\rho}{(1+\rho)^{3/2}} z_{\gamma}$
4. Як $k < Crit$ обираємо $H_0$, інакше $H_1$


In [4]:
# example 1
X = np.random.exponential(1, 10)
Y = np.random.exponential(1, 20)
intervals = []
for i in range(1, len(X + 1)):
    intervals.append([X[i - 1], X[i]])

res = np.zeros(len(intervals))
for i, x in enumerate(intervals):
    for item in Y:
        if intervals[i][0] <= item <= intervals[i][1]:
            print(f"{intervals[i][0]} <= {item} <= {intervals[i][1]}")
            res[i] += 1
    if res[i] != 0:
        print(f"{i} :: {res[i]}")
        print("+++++++++++++++++++++++++")

print()
print()
print(res)
print("+" * 20)
print(np.count_nonzero(res == 0))


0.022560150383076922 <= 0.11194515924601654 <= 2.9367385911541155
0.022560150383076922 <= 0.8694823233980112 <= 2.9367385911541155
0.022560150383076922 <= 0.40725772879269456 <= 2.9367385911541155
0.022560150383076922 <= 0.39779858599467083 <= 2.9367385911541155
0.022560150383076922 <= 0.5555465960559985 <= 2.9367385911541155
0.022560150383076922 <= 0.797720282201548 <= 2.9367385911541155
0.022560150383076922 <= 1.3457958010976374 <= 2.9367385911541155
0.022560150383076922 <= 0.16042711281665994 <= 2.9367385911541155
0.022560150383076922 <= 0.12613154768249943 <= 2.9367385911541155
0.022560150383076922 <= 2.0827586434585923 <= 2.9367385911541155
0.022560150383076922 <= 0.15755769934349198 <= 2.9367385911541155
0.022560150383076922 <= 1.9549459578991897 <= 2.9367385911541155
0.022560150383076922 <= 1.2999796662526257 <= 2.9367385911541155
0.022560150383076922 <= 0.04473339939838998 <= 2.9367385911541155
0.022560150383076922 <= 0.4836437121237178 <= 2.9367385911541155
0.02256015038307692

In [5]:
NM = [[500, 1000], [5000, 10000]]  #, [50000, 100000]]

xi_empty = []
crit_empty = []
RES_empty = []
for nm in NM:

    n = nm[0]
    m = nm[1]
    ro = m / n

    X = np.random.exponential(1, n)
    Y = np.random.exponential(1, m)

    # робимо інтервали [1 коорд, 2 коорд]
    intervals = []
    for i in range(1, len(X + 1)):
        intervals.append([X[i - 1], X[i]])

    # рахуємо кількість попадань у відрізок
    res = np.zeros(len(intervals))
    for i, x in enumerate(intervals):
        for item in Y:
            if intervals[i][0] <= item <= intervals[i][1]: res[i] += 1

    xi_empty_temp = np.count_nonzero(res == 0)
    xi_empty.append(xi_empty_temp)

    crit = n / (1 + ro) + np.math.sqrt(n) * ro * z_gamma_empty / (1 + ro) ** (3 / 2)
    crit_empty.append(crit)
    if xi_empty_temp < crit:
        RES_empty.append("H0")
    else:
        RES_empty.append("False")

data_empty = {"xi_empty (n, m)": xi_empty, f"crit": crit_empty, "Result": RES_empty}
df_empty = pd.DataFrame.from_dict(data_empty, orient='index', columns=[f"n::{item[0]}; m::{item[1]}" for item in NM])


In [6]:
df_empty

,n::500; m::1000,n::5000; m::10000
"xi_empty (n, m)",256,2538
crit,167.206361,1668.373329
Result,False,False


## Завдання 2

### Ранжування

In [7]:
rank = lambda data: pd.Series(data).rank(method="min").to_list()
gamma = 0.05
z_gamma = 0.0627

### Критерій перевірки гіпотези незалежності $H_0$

1) Маємо вибірку $(\overline{X}, \overline{Y}) = { (X_1, Y_1), \cdots (X_n, Y_n) }$
2) Обчислюємо $R_i$ -- ранг $X_i$ серед елементів $X_1, \cdots, X_n$
3) Обчислюємо $S_i$ -- ранг $Y_i$ серед елементів $Y_1, \cdots, Y_n$
4) Маємо пари $(\overline{R}, \overline{S}) = { (R_1, S_1), \cdots (R_n, S_n) }$
5) Сортуємо за $R_i \Rightarrow (1, V_1), \cdots (n, V_n)$
6) Рахуємо статистику $\rho_n(\bar X, \bar Y) =  1 - \frac{6}{n (n^{2} - 1} \cdot \sum\limits_{i \in [1,n]} (i - V_{i}^{2})$ (коеф кореляції двох множин рангів)
7) Порівнюємо $\rho_n \bigvee \dfrac{z_\gamma}{\sqrt{n}}$, як $<$ приймаємо $H_0$, інакше $H_1$

### критей Спірмана

In [8]:
def SpearMan(V):
    n = len(V)
    return 1 - 6 * np.sum(pow(V[:, 0] - V[:, 1], 2)) / (n * (pow(n, 2) - 1))
    ...

а) $Y_i = \ksi_$

In [18]:
N_arr = [500, 5000, 50000]

z_gamma_spearman = [z_gamma / np.math.sqrt(n) for n in N_arr]

RHO = []
Hyp_Spearman = []
for i, n in enumerate(N_arr):
    data_xi = np.random.uniform(0, 1, n)
    data_nu = np.random.uniform(-1, 1, n)
    X = rank(data_xi)
    Y = rank(data_xi * data_nu)

    res = np.array([X, Y])
    # паруємо
    res = np.transpose(res)
    # сортуємо по Х
    res = np.argsort(res, 0)

    # обчислюємо статистику
    test = abs(SpearMan(res))
    RHO.append(test)
    if test < z_gamma_spearman[i]:
        Hyp_Spearman.append("H0")
    else:
        Hyp_Spearman.append("False")

data_Spearman = {"Spearman statistic": RHO, f"z_gamma": z_gamma_spearman, "Result": Hyp_Spearman}
df_Spearman = pd.DataFrame.from_dict(data_Spearman, orient='index', columns=N_arr)

df_Spearman

,500,5000,50000
Spearman statistic,0.065808,0.009156,0.002839
z_gamma,0.002804,0.000887,0.00028
Result,False,False,False


### критерій Кендала

### для обчислення гіпотези використовується $\tau = \frac{4N}{n(n-1)} - 1$

In [10]:
N_arr = [500, 5000, 50000]

z_gamma_kendal = [z_gamma / np.math.sqrt(n) / 3 * 2 for n in N_arr]
tau = []
Hyp_Kendal = []
for tmp, n in enumerate(N_arr):
    data_xi = np.random.uniform(0, 1, n)
    data_nu = np.random.uniform(-1, 1, n)
    X = rank(data_xi)
    Y = rank(data_xi * data_nu)

    res = np.array([X, Y], np.int64)
    res = np.transpose(res)
    res = res[res[:, 0].argsort()]
    P_q = 0
    for i in res[1:-1, 0]:
        if res[i, 1] > res[i - 1, 1]: P_q += res[i, 1] - res[i - 1, 1]

    P_p = n * (n - 1) / 2 - P_q
    res = 4 * P_p / (n * (n - 1)) - 1
    tau.append(res)
    if res < z_gamma_kendal[tmp]:
        Hyp_Kendal.append("H0")
    else:
        Hyp_Kendal.append("false")

data_Kendal = {"Kendal statistic": tau, f"z_gamma": z_gamma_kendal, "Result": Hyp_Kendal}
df_Kendal = pd.DataFrame.from_dict(data_Kendal, orient='index', columns=N_arr)

df_Kendal

,500,5000,50000
Kendal statistic,0.3878,0.392356,0.390492
z_gamma,0.001869,0.000591,0.000187
Result,false,false,false


### для в.в. $\xi$ та $\eta$

In [11]:
#для прикладу
data_xi = [100, 118, 112, 97, 99, 103, 102, 132, 122, 121, 115, 117, 109, 111]
data_eta = [154, 123, 120, 213, 200, 187, 155, 100, 114, 115, 107, 176, 143, 111]
n = len(data_xi)
X = rank(data_xi)
Y = rank(data_eta)

res = np.array([X, Y], np.int64)
res = np.transpose(res)
res = res[res[:, 0].argsort()]
P_q = 0
for i in res[1:-1, 0]:
    if res[i, 1] > res[i - 1, 1]: P_q += res[i, 1] - res[i - 1, 1]

P_p = n * (n - 1) / 2 - P_q
print('P_q = ', P_q, '\n', 'P_p = ', P_p, '\n', 4 * P_q / (n * (n - 1)) - 1)


P_q =  15 
 P_p =  76.0 
 -0.6703296703296704


### Завдання 3
#### Критерій інверсій

У даному методі будемо перевіряти гіпотезу випадковості (повного хаосу), що означає:
$$H_0: F_{\overline{x}}(u_1, \ldots, u_n) = F_{\xi}(u_1) \cdot F_{\xi}(u_2) \cdot \ldots \cdot F_{\xi}(u_n)$$

**Алгоритм обчислення:**
1. Згенерувати $n$ спостережень
2. Обчислюємо "хаос" $k$ у варіаційному ряді,
3. Обчислюємо критичну область $\dfrac{z_{\gamma} n \sqrt{n}}{6}$
4. Рахуємо статистику $S_n(\overline{X}) = \Big| k - \dfrac{n(n-1)}{4}  \Big|$
5. Порівнюємо $ \Big| k - \dfrac{n(n-1)}{4}  \Big| \bigvee  \dfrac{z_{\gamma} n \sqrt{n}}{6}$
як $<$, приймаємо гіпотезу $H_0$, інакше -- ні

In [14]:
N_arr = [500, 5000]  #, 50000]
z_gamma_inv = [z_gamma / 6 * n ** (3 / 2) for n in N_arr]
LAMBDA = []
RES_inv = []

def genX(n):
    X = []
    ksi = np.random.uniform(-1, 1, n)
    for j in range(n):
      x=np.sum(ksi[:j+1])/(j+1)
      X.append(x)
    return X

for tmp, n in enumerate(N_arr):
    X = genX(n)
    k = 0

    for x in range(1, len(X) - 1):
        for j in range(x, len(X)):
            i = j - 1
            if (X[j] < X[i]) and (i < j): k += 1

    print(f"k_{tmp}: {k}")

    statistic = (abs(k - n * (n - 1) / 4))
    LAMBDA.append(statistic)
    if statistic < z_gamma_inv[tmp]:
        RES_inv.append("H0")
    else:
        RES_inv.append("false")

data_inv = {"LAMBDA": LAMBDA, f"z_gamma": z_gamma_inv, "Result": RES_inv}
df_inv = pd.DataFrame.from_dict(data_inv, orient='index', columns=N_arr)

df_inv

k_0: 60029
k_1: 6145292


,500,5000
LAMBDA,2346.0,103458.0
z_gamma,116.834552,3694.632932
Result,false,false
